In [2]:
import scqubits as scq
import numpy as np
from systemclass import SNAIL
def sort_eigenpairs(eigenvalues, eigenvectors):
    n = eigenvectors.shape[0]
    sorted_indices = []

    for i in range(n):
        max_abs_vals = np.abs(eigenvectors[i, :])
        max_index = np.argmax(max_abs_vals)
        while max_index in sorted_indices:
            max_abs_vals[max_index] = -np.inf
            max_index = np.argmax(max_abs_vals)
        sorted_indices.append(max_index)

    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    return sorted_eigenvalues, sorted_eigenvectors
d1=d2=2
fluxonium1 = scq.Fluxonium(
    EJ=4.6,
    EC=0.9,
    EL=0.21,
    cutoff = 110,
    flux = 0.5,
    truncated_dim=d1
)
fluxonium2 = scq.Fluxonium(
    EJ=5.5,
    EC=0.9,
    EL=0.21,
    cutoff = 110,
    flux = 0.5,
    truncated_dim=d2
)
# EJ = 80*2*np.pi
# EC = 0.2*2*np.pi
# beta = 0.15
# ng = 0
# ncut = 100
# flux = 0.22
# N = 3
# dims = 10

# # truncated_dim:int = 6
# snail = SNAIL(EJ,EC,beta,ng,ncut,flux,N,dims)
# Hs , charge_op = snail.spectrum_charge()

In [3]:
def annihilation(dim):
    return np.diag(np.sqrt(np.arange(1,dim)),1)
def creation(dim):
    return np.diag(np.sqrt(np.arange(1,dim)),-1)
dims = 10
sd = creation(dims)
s = annihilation(dims)
omega_s = 5*2*np.pi
g3 = 0.05*2*np.pi
Hs = omega_s*sd@s
Is = 1*np.identity(dims)

H3 = g3*(s-Is+sd-Is)@(s-Is+sd-Is)@(s-Is+sd-Is)
charge_op = -1j*(s-sd)

In [4]:
phi1 = fluxonium1.phi_operator(energy_esys=True)
phi2 = fluxonium2.phi_operator(energy_esys=True)

In [5]:
H1 = fluxonium1.hamiltonian(energy_esys=True)*2*np.pi
H2 = fluxonium2.hamiltonian(energy_esys=True)*2*np.pi
H1 = H1-np.identity(d1)*H1[0,0]
H2 = H2-np.identity(d2)*H2[0,0]

In [8]:
If1 = np.identity(d1)
If2 = np.identity(d2)
Is = np.identity(dims)

In [9]:
Hs = np.kron(np.kron(Hs,If1),If2)
H3 = np.kron(np.kron(H3,If1),If2)
H1 = np.kron(np.kron(Is,H1),If2)
H2 = np.kron(np.kron(Is,If1),H2)

In [20]:
g_int1 = 0.13*2*np.pi
g_int2 = 0.13*2*np.pi
H_int1 = g_int1 * np.kron(np.kron(charge_op,phi1),If2) 
H_int2 = g_int2 * np.kron(np.kron(charge_op,If1),phi2)

In [21]:
H = Hs + H1 + H2 + H_int1 + H_int2
Hc  =  np.kron(np.kron(charge_op,If1),If2)
energy0,U = np.linalg.eigh(H)
energy0,U = sort_eigenpairs(energy0, U)
Ud = U.transpose().conjugate()
H = Ud@H@U
H3 = Ud@H3@U
Hc = Ud@Hc@U

In [22]:
(energy0 - energy0[0])/2/np.pi

array([0.00000000e+00, 1.64321328e-02, 1.36891745e-01, 1.53323249e-01,
       5.00072937e+00, 5.01648506e+00, 5.13683985e+00, 5.15259368e+00,
       1.00014398e+01, 1.00165363e+01, 1.01367896e+01, 1.01518831e+01,
       1.50021314e+01, 1.50165858e+01, 1.51367411e+01, 1.51511913e+01,
       2.00028044e+01, 2.00166336e+01, 2.01366941e+01, 2.01505182e+01,
       2.50034591e+01, 2.50166803e+01, 2.51366487e+01, 2.51498637e+01,
       3.00041387e+01, 3.00167730e+01, 3.01366053e+01, 3.01492325e+01,
       3.50077320e+01, 3.50200969e+01, 3.51365895e+01, 3.51489478e+01,
       4.00826277e+01, 4.01104668e+01, 4.01397617e+01, 4.01676109e+01,
       4.51336815e+01, 4.51356347e+01, 4.60983182e+01, 4.61002842e+01])

In [23]:
U[3]

array([-4.17678143e-01-1.06988269e-20j, -1.22679644e-14+4.45823933e-16j,
       -2.05391260e-15+4.85722573e-17j, -9.02018157e-01-1.63375390e-20j,
        1.54043445e-15-2.06432094e-16j, -1.07925651e-01-4.24659461e-19j,
        1.10253716e-02-7.89578186e-18j,  9.78384040e-16-1.55257751e-16j,
       -1.11357171e-02-3.89528410e-19j, -8.77770079e-16+2.29850861e-17j,
       -3.75405002e-16-4.43845264e-19j,  3.69021326e-03-7.41284063e-19j,
        5.07948718e-17-5.25838054e-18j, -1.03481066e-03+1.26999393e-20j,
        9.22862680e-05+9.57591125e-20j,  2.03287907e-17+3.57786717e-18j,
       -7.61433681e-05-9.85589692e-19j, -4.09625133e-18+8.06375366e-19j,
       -2.96461532e-20+3.81164826e-20j,  2.38362199e-05+1.30378325e-19j,
        1.91852963e-19+8.47032947e-21j, -5.45954125e-06+8.99271829e-19j,
        4.59383684e-07-1.70254484e-19j, -2.25522522e-20+4.46346659e-21j,
       -3.29499620e-07+3.92458186e-19j, -4.89690923e-21+5.10124869e-22j,
       -3.93737972e-22+6.89888018e-23j,  9.79764894

In [24]:
np.abs(H3[0:4,0:4])

array([[4.50583952e+00, 1.24102008e+00, 1.44011691e-01, 3.67536028e-03],
       [1.24102008e+00, 4.40278348e+00, 5.58902042e-03, 4.14016453e-01],
       [1.44011691e-01, 5.58902042e-03, 4.39808190e+00, 5.88038151e-02],
       [3.67536028e-03, 4.14016453e-01, 5.88038151e-02, 4.38029573e+00]])

In [25]:
Hc[0:4,0:4]

array([[-4.40034408e-15-7.16604610e-34j,  2.87579384e-01-1.08056112e-02j,
        -2.80139607e-02+8.51782299e-04j, -2.01928662e-15+6.31682997e-17j],
       [ 2.87579384e-01+1.08056112e-02j,  5.20572545e-15-5.86446045e-33j,
         1.84268374e-16+3.21894556e-16j,  1.03515131e-01+3.88951476e-03j],
       [-2.80139607e-02-8.51782299e-04j,  1.84268374e-16-3.21894556e-16j,
        -5.23577748e-16-2.08334121e-32j, -1.23073825e-02-3.74213794e-04j],
       [-2.01928662e-15-6.31682997e-17j,  1.03515131e-01-3.88951476e-03j,
        -1.23073825e-02+3.74213794e-04j, -9.59210115e-16+3.90517380e-33j]])

In [30]:
np.diag(H)/2/np.pi - (np.diag(H)/2/np.pi)[0]

array([0.00000000e+00+0.00000000e+00j, 7.43551710e-03+2.40663628e-19j,
       2.24023997e-01+1.54195693e-18j, 2.31025421e-01-2.35423674e-33j,
       7.99595604e-01+7.71693486e-18j, 8.04056326e-01-1.11635509e-31j,
       1.01956578e+00-3.90956183e-30j, 1.02294572e+00-7.06891662e-18j,
       1.59779881e+00+3.57259230e-32j, 1.60031968e+00-1.18717446e-17j,
       1.81519647e+00+5.24688399e-18j, 1.81635885e+00-2.43782435e-31j,
       2.39497130e+00-5.75392124e-17j, 2.39634269e+00+7.92814591e-33j,
       2.61086707e+00+1.43307629e-16j, 2.61089098e+00-3.36802731e-30j,
       3.19165866e+00-4.07227501e-32j, 3.19244811e+00-1.94149208e-17j,
       3.40613682e+00+1.80922925e-31j, 3.40662994e+00+3.66873645e-17j,
       3.99195883e+00+4.09806590e-19j, 3.99254065e+00-1.89924274e-32j,
       4.20190758e+00-2.65632106e-17j, 4.20239916e+00+3.48144913e-30j,
       4.82355378e+00-2.11529080e-32j, 4.82408895e+00+6.83994222e-18j,
       4.99795253e+00+1.84291858e-30j, 4.99818655e+00-8.80951423e-18j,
      